<a href="https://colab.research.google.com/github/stevejj4/Machine-Learning/blob/main/Insurance_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install google-cloud-bigquery pandas


In [2]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd

# Replace with your project ID
project_id = 'river-messenger-430112-e1'

# Initialize BigQuery client
client = bigquery.Client(project=project_id)


In [3]:
import gspread
from google.auth import default

# Authenticate and create the gspread client
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheets by title
worksheet_customers = gc.open('Insurance_data').worksheet('Customers')
worksheet_policies = gc.open('Insurance_data').worksheet('Policies')
worksheet_interactions = gc.open('Insurance_data').worksheet('Interactions')
worksheet_claims = gc.open('Insurance_data').worksheet('Claims')

# Load the data into pandas DataFrames
customers_df = pd.DataFrame(worksheet_customers.get_all_records())
policies_df = pd.DataFrame(worksheet_policies.get_all_records())
interactions_df = pd.DataFrame(worksheet_interactions.get_all_records())
claims_df = pd.DataFrame(worksheet_claims.get_all_records())


In [5]:
def upload_to_bigquery(df, table_name):
    dataset_id = 'Insurance_data'  # Correct dataset ID
    # Use the `TableReference` class to construct the table ID in the correct format
    table_ref = bigquery.TableReference(
        bigquery.DatasetReference(project_id, dataset_id), table_name
    )

    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    )

    job = client.load_table_from_dataframe(
        df, table_ref, job_config=job_config
    )

    job.result()  # Wait for the job to complete

    print(f"Loaded {job.output_rows} rows into {table_ref}.")

# Upload each DataFrame to BigQuery
upload_to_bigquery(customers_df, 'customers')
upload_to_bigquery(policies_df, 'policies')
upload_to_bigquery(interactions_df, 'interactions')
upload_to_bigquery(claims_df, 'claims')

Loaded 2004 rows into river-messenger-430112-e1.Insurance_data.customers.
Loaded 2004 rows into river-messenger-430112-e1.Insurance_data.policies.
Loaded 5000 rows into river-messenger-430112-e1.Insurance_data.interactions.
Loaded 3000 rows into river-messenger-430112-e1.Insurance_data.claims.
